In [2]:
import requests
import numpy as np
import pandas as pd
import os
import time
import datetime
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import numpy as np

In [13]:
col_dict = {"corp_id": str, "corp_name": str, "website": str}
corp = pd.read_excel('C:/Users/user/Downloads/corp_website_done.xlsx', dtype=col_dict)

In [15]:
corp['corp_id']

0        000036QI
1        000040BA
2        000080MI
3        000195PH
4        000259PT
           ...   
12686    98910003
12687    97413338
12688    97397885
12689    97417474
12690    97498531
Name: corp_id, Length: 12691, dtype: object

In [16]:
corp['corp_id'] = corp['corp_id'].map(lambda x:("0"*(8-len(x))+x) if len(x)<8 else x)

In [17]:
corp_id_list_total = list(corp["corp_id"])

In [18]:
corp

,corp_id,corp_name,corp_name_web,corp_name_web_eng,website,check,time_out
0,000036QI,NaN,NaN,NaN,NaN,N,NaN
1,000040BA,NaN,NaN,NaN,NaN,N,NaN
2,000080MI,NaN,NaN,NaN,NaN,N,NaN
3,000195PH,NaN,NaN,NaN,NaN,N,NaN
4,000259PT,NaN,NaN,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...
12686,98910003,NaN,NaN,NaN,NaN,NaN,NaN
12687,97413338,NaN,NaN,NaN,NaN,NaN,NaN
12688,97397885,NaN,NaN,NaN,NaN,NaN,NaN
12689,97417474,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#selenium settings
options = Options()
options.add_argument("--disable-gpu");
chrome = webdriver.Chrome('./chromedriver',options=options)
#設定等待時間
chrome.implicitly_wait(5)
chrome.set_page_load_timeout(15)

In [20]:
corp_id_list = corp_id_list_total[1980:3000]

In [21]:
corp_id_list

['16441221',
 '16299462',
 '18045198',
 '16238959',
 '16631236',
 '183545IN',
 '194782LI',
 '17657846',
 '16620462',
 '18051971',
 '16713911',
 '16248540',
 '16860552',
 '16584923',
 '168006VI',
 '16785689',
 '16886568',
 '186375SE',
 '188117NE',
 '16952721',
 '196812BR',
 '16175862',
 '18880007',
 '16602464',
 '168338ZH',
 '16804264',
 '16821046',
 '16275757',
 '19084015',
 '16663216',
 '16374969',
 '16697102',
 '16319002',
 '16727175',
 '16959554',
 '16832064',
 '16828358',
 '16436873',
 '16163161',
 '16645983',
 '16959935',
 '16759845',
 '16845455',
 '16218145',
 '16804406',
 '16887406',
 '18257848',
 '16505809',
 '16756381',
 '194962SA',
 '186379SH',
 '16654687',
 '16733769',
 '19812062',
 '16851218',
 '170267SE',
 '17521089',
 '16839714',
 '16366980',
 '16619628',
 '16610020',
 '16169215',
 '19386623',
 '18255428',
 '16617375',
 '16291320',
 '16706307',
 '16262562',
 '16664777',
 '18795706',
 '16585704',
 '16880136',
 '16314864',
 '188327MI',
 '16378536',
 '17192137',
 '174472BM',

In [23]:
#loop
counter=1
for id in corp_id_list:
    try:
        chrome.get("https://www.twincn.com/item.aspx?no="+str(id))
        try:
            table_tr_list = chrome.find_element("class name", "table-responsive").find_elements("tag name", "tr")
            table_list = []
            for tr in table_tr_list:
                #將每一個tr的資料根據td查詢出來，返回結果為list物件
                table_td_list = tr.find_elements("tag name", "td")
                row_list = []
                for td in table_td_list:    #遍歷每一個td
                    row_list.append(td.text)    #取出表格的資料，並放入行列表裡
                table_list.append(row_list)
            for i in table_list:
                if i[0]=='網址':
                    corp.loc[corp["corp_id"]==id,"website"] = i[1]
                elif i[0]=='公司名稱':
                    corp.loc[corp["corp_id"]==id,"corp_name_web"] = i[1]
                elif i[0]=='英文名稱':
                    corp.loc[corp["corp_id"]==id,"corp_name_web_eng"] = i[1]      
                corp.loc[corp["corp_id"]==id,"check"] = "Y"
        except:
            corp.loc[corp["corp_id"]==id,"check"] = "N"
        rand_time = np.random.randint(15)
        time.sleep(rand_time)  #強制等待
        counter = counter+1
        if counter%50==0:
            print("已完成{}筆\n".format(counter))
            corp.to_excel('C:/Users/user/Downloads/corp_website_done.xlsx',index=False)
    except:
        corp.loc[corp["corp_id"]==id,"time_out"] = "Y"

已完成50筆

已完成100筆

已完成150筆

已完成200筆

已完成250筆

已完成300筆



In [16]:
corp

,corp_id,corp_name,corp_name_web,corp_name_web_eng,website,check
0,22099131,台灣積體電路製造股份有限公司,台灣積體電路製造股份有限公司,Taiwan Semiconductor Manufacturing Company Lim...,https://www.tsmc.com,Y
1,01615407,全翔食品行,全翔食品行,NaN,NaN,Y
2,7788,NaN,NaN,NaN,NaN,N
3,12336458,NaN,NaN,NaN,NaN,N
4,8046542,NaN,NaN,NaN,NaN,N
5,84149961,NaN,聯發科技股份有限公司,MEDIATEK INC.,http://www.mediatek.com,Y


In [122]:
corp.to_excel('corp_website_done.xlsx',index=False)